# TD 16 - Expressions arithmétiques

## Partie 1 - Arbre d'une expression arithmétique

In [7]:
type op =
  | Plus
  | Fois
  | Moins

type expr =
  | C of int
  | N of op * expr * expr

type op = Plus | Fois | Moins


type expr = C of int | N of op * expr * expr


### Exercice 1

In [23]:
let exemple = N ( Plus, N ( Fois, C 4 ,N ( Moins, C 8,C 9)), N ( Plus, C 6,C 7))
let m,q = (N (Plus,C 2 ,N(Fois, C 3, C 4)),N (Fois,N(Plus, C 2, C 3),C 4))

val exemple : expr =
  N (Plus, N (Fois, C 4, N (Moins, C 8, C 9)), N (Plus, C 6, C 7))


val m : expr = N (Plus, C 2, N (Fois, C 3, C 4))
val q : expr = N (Fois, N (Plus, C 2, C 3), C 4)


### Exercice 2

In [15]:
let applique op a b = match op with
    |Fois -> a*b
    |Plus -> a+b
    |Moins -> a-b

val applique : op -> int -> int -> int = <fun>


In [16]:
let rec eval arbre = match arbre with
    |C w -> w
    |N (op,a,b) -> applique op (eval a) (eval b)

val eval : expr -> int = <fun>


In [17]:
assert (eval exemple = 9)

- : unit = ()


## Parties 2 : Différentes notations

### Exercice 3

4*8+9+6+7 pour infixe, préfixe on a +*4-89+67, postfixe ++76*-984

### Exercice 4

papier, je recopierai un jour

### Exercice 5

In [27]:
type lexeme = PO | PF | Op of op | Val of int

type lexeme = PO | PF | Op of op | Val of int


In [31]:
let rec prefixe arbre = match arbre with
    |C n -> [Val n]
    |N (x,g,d) ->[PO] @ [Op x] @ prefixe g @ prefixe d @ [PF]

val prefixe : expr -> lexeme list = <fun>


In [33]:
let rec postfixe arbre = match arbre with
    |C n -> [Val n]
    |N (x,g,d) ->[PF]  @ prefixe g @ prefixe d @ [Op x] @[PO]

val postfixe : expr -> lexeme list = <fun>


In [36]:
let rec infixe arbre = match arbre with
    |C n -> [Val n]
    |N (x,g,d) ->[PO]  @ prefixe g @ [PF ] @ [Op x] @ [PO]@ prefixe d @ [PF]

val infixe : expr -> lexeme list = <fun>


### Exercice 6

In [39]:
let eval_post u = 
    let rec aux u v = match u with
        |[] -> v
        |x::r -> match x with
            |Val w-> aux r ( w::v)
            |Op op -> match v with
                |a::b::xs -> aux r ((applique op a b)::v)
                |_ -> failwith "error"
    in List.hd (aux u []) ;;

(* on a supposé qu'il n'y avait pas de parenthèses)

File "[39]", line 4, characters 17-196:
4 | .................match x with
5 |             |Val w-> aux r ( w::v)
6 |             |Op op -> match v with
7 |                 |a::b::xs -> aux r ((applique op a b)::v)
8 |                 |_ -> failwith "error"
Warning 8: this pattern-matching is not exhaustive.
Here is an example of a case that is not matched:
(PO|PF)


val eval_post : lexeme list -> int = <fun>


### Exercice 7

In [43]:
let arbre_of_post u = 
    let rec aux u v = match u with
        |[] -> v
        |x::r -> match x with
            |Val w -> aux r ((C w)::v)
            |Op op -> match v with
                |a::b::xs -> aux r ((N (op,a,b))::v) (* c'est bien le bon ordre*)
                |_-> failwith "error"
    in List .hd (aux u []) ;;

File "[43]", line 4, characters 17-194:
4 | .................match x with
5 |             |Val w -> aux r ((C w)::v)
6 |             |Op op -> match v with
7 |                 |a::b::xs -> aux r ((N (op,a,b))::v)
8 |                 |_-> failwith "error"
Warning 8: this pattern-matching is not exhaustive.
Here is an example of a case that is not matched:
(PO|PF)


val arbre_of_post : lexeme list -> expr = <fun>


## Partie 3 - Expressions avec variables

In [44]:
type expr2 =
  | N of op * expr2 * expr2
  | C of int
  | V of int

type expr2 = N of op * expr2 * expr2 | C of int | V of int


### Exercice 8

In [45]:
let rec max_var expr = match expr with
    |C n -> min_int
    |V n -> n
    |N (op,g,d) -> max (max_var g) (max_var d) ;;

val max_var : expr2 -> int = <fun>


In [46]:
let rec eval_contexte expr v = let i = max_var expr in
    match expr with
    |V k -> v.(k)
    |C n  -> n
    |N (op,g,d) -> applique op (eval_contexte g v ) (eval_contexte d v ) ;;

val eval_contexte : expr2 -> int array -> int = <fun>


### Exercice 9

dans ce cas-ci, l'expression n'est constitué que de noeuds et de feuilles donnée par le constructeur C

In [51]:
let rec eval_partielle expr v = match expr with
    |C n -> C n
    |V k -> if k < Array.length v then C v.(k) else V k
    | N (op,g,d) -> N (op, eval_partielle g v, eval_partielle d v)

val eval_partielle : expr2 -> int array -> expr2 = <fun>


## Partie 4 - Forme normale pour l'associativité

In [52]:
type op = Plus | Fois

type expr3 =
  | C of int
  | V of int
  | N of op * expr3 * expr3

type expr_naire =
  | Cn of int
  | Vn of int
  | Nn of op * expr_naire list

type op = Plus | Fois


type expr3 = C of int | V of int | N of op * expr3 * expr3


type expr_naire = Cn of int | Vn of int | Nn of op * expr_naire list


### Exercice 10

In [81]:
let recup1 a = match a with
    |N (op,g,d) -> Some op
    |_ -> None

let recup2 a = 
    match a with
    |Nn (op,u) ->u
    |_ -> []

let remet a=
    match a with
        |C n -> Cn n
        |V n -> Vn n
        
let rec normalise expr = match expr with
    |C n -> Cn n
    |V n -> Vn n
    |N (op,g,d) -> match recup1 g,recup1 d with
        |Some x, Some y when x=y && x=op -> let p,q = normalise g, normalise d in Nn (op, (recup2 p) @ (recup2 q)) (* le type est bien une liste car on a un noeud*)
        |Some x,Some y -> Nn (op,[normalise g;normalise d])
        |None, Some y -> let p = normalise d in Nn (op , (remet g)::(recup2 p))
        |Some x, None -> let p = normalise g in Nn (op, (remet d)::(recup2 p))
        |None , None -> Nn (op, [remet g; remet d])

val recup1 : expr3 -> op option = <fun>


val recup2 : expr_naire -> expr_naire list = <fun>


File "[81]", line 11, characters 4-58:
11 | ....match a with
12 |         |C n -> Cn n
13 |         |V n -> Vn n
Warning 8: this pattern-matching is not exhaustive.
Here is an example of a case that is not matched:
N (_, _, _)


val remet : expr3 -> expr_naire = <fun>


val normalise : expr3 -> expr_naire = <fun>


In [80]:
let a_normaliser =
  N (Plus, 
     N (Plus, 
        N (Plus, 
           V 3, 
           V 4), 
        N (Fois, 
           V 5, 
           V 6)), 
     N (Fois, 
        N (Plus,
           V 7,
           N (Fois,
              V 8,
              V 9)),
        N (Fois,
           V 11,
           V 12))) ;;

normalise a_normaliser

val a_normaliser : expr3 =
  N (Plus, N (Plus, N (Plus, V 3, V 4), N (Fois, V 5, V 6)),
   N (Fois, N (Plus, V 7, N (Fois, V 8, V 9)), N (Fois, V 11, V 12)))


- : expr_naire =
Nn (Plus,
 [Nn (Plus, [Nn (Plus, [Vn 3; Vn 4]); Nn (Fois, [Vn 5; Vn 6])]);
  Nn (Fois, [Nn (Plus, [Vn 7; Vn 8; Vn 9]); Nn (Fois, [Vn 11; Vn 12])])])


In [78]:
assert (normalise a_normaliser = 
    Nn (Plus, [Vn 3; 
               Vn 4; 
               Nn (Fois, [Vn 5; Vn 6]);
               Nn (Fois, [Nn (Plus, [Vn 7; 
                                     Nn (Fois, [Vn 8; Vn 9])]); 
                          Vn 11; 
                          Vn 12])]))

error: runtime_error